<a href="https://colab.research.google.com/github/ratral/hyd4gpv_py/blob/main/gpv_operation_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Loading the necessary libraries and functions.

In [ ]:
#@title 0.1 Libreries/Packages
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

from scipy.optimize import fsolve
from numpy.core.fromnumeric import mean

# Mark the versions
#pd.show_versions()

In [ ]:
#@title Dealing with the Paths
print(f"Current working directory: {Path.cwd()}")
print(f"Home directory: {Path.home()}")

path_valves_data = Path("/content/drive/MyDrive/Colab Notebooks/gpv_data.csv") 
print(path_valves_data.exists())

Current working directory: /content
Home directory: /root
True


In [ ]:
#@title 0.2 Vectorized Functions
"""
Created on Sun Sep 25 16:19:54 2022
@author: raul
"""

import numpy as np
from scipy.optimize import fsolve

def atm_pressure(masl) -> float:
  """Returns: Atmosphere  Pressure in [bar]"""
  return (1/1000)*((44331.514 - masl)/11880.516)**(1/0.1902632)
v_atm_pressure = np.vectorize(atm_pressure)

def vapor_pressure(temp_c) -> float:
  """Returns: Vapor pressure [bar]"""
  return (0.61121*np.exp((18.678-temp_c/234.5)*(temp_c/(257.14+temp_c))))/100
v_vapor_pressure = np.vectorize(vapor_pressure)

def density(temp_c) -> float:
  """Returns: density of water in [kg/m³]"""
  temp_k = temp_c + 273.15
  a = 0.14395; b = 0.0112; c = 649.727; d = 0.05107
  return  a/(b**(1 + (1-temp_k/c)**d))
v_density = np.vectorize(density)

def velocity(flow, diameter) -> float:
  """Returns: Velocity in [m/s]"""
  return (flow/3600)/((np.pi*diameter**2)/4)
v_velocity = np.vectorize(velocity)

def velocity_factor(flow, diameter) -> float:
  """Returns: Velocity factor in [bar]"""
  return (v_velocity(flow, diameter)**2 / (2*9.80665)) / 10
v_velocity_factor = np.vectorize(velocity_factor)

def absolute_pressure(gauge_pressure, masl) -> float:
  """Returns: Absolute Pressure in [bar]"""
  ap = v_atm_pressure(masl)
  return gauge_pressure + ap
v_absolute_pressure = np.vectorize(absolute_pressure)

def sigma_0(p_up, p_down, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  return (p1 - v_vapor_pressure(temp_c))/(p1-p2)
v_sigma_0 = np.vectorize(sigma_0)

def sigma_1(p_up, p_down, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  return (p2 - v_vapor_pressure(temp_c))/(p1-p2)
v_sigma_1 = np.vectorize(sigma_1)

def sigma_2(p_up, p_down, flow, diameter, masl, temp_c) -> float:
  p1 = v_absolute_pressure(p_up, masl)
  p2 = v_absolute_pressure(p_down, masl)
  v_factor = velocity_factor(flow, diameter)
  return (p2 - v_vapor_pressure(temp_c))/(p1 - p2 + v_factor)
v_sigma_2 = np.vectorize(sigma_2)

def flow_coefficent(p_up, p_down, flow, temp_c) -> float:
  ''' Return Flow Coefficent Kv in m3/h'''
  return flow*np.sqrt((v_density(temp_c)/1000)/(p_up - p_down))
v_flow_coefficent = np.vectorize(flow_coefficent)

def drop_coefficient(p_up, p_down, flow, diameter, temp_c) -> float:
  ''' Return Pressure Drop Coefficient - zeta'''
  kv = v_flow_coefficent(p_up, p_down, flow, temp_c)
  return (1/626.3)*((diameter*1000)**2/kv)**2
v_drop_coefficient = np.vectorize(drop_coefficient)

def kv_fun_zeta(diameter, zeta_value) -> float:
  ''' Return Flow Coefficent Kv in m3/h'''
  return ((diameter*1000)**2)/np.sqrt(626.3*zeta_value)
v_kv_fun_zeta = np.vectorize(kv_fun_zeta)

#-------------------------------------------------------------------------------
def resistance_coefficient(diameter, dn_up, dn_down) -> float:
  '''Return the Resistance coefficients of all fittings attached to the control valve'''
  diameter *= 1000; dn_up *= 1000; dn_down *= 1000
  reducer  =  0.5 * ((1-(diameter/dn_up)**2)**2)
  diffuser =  ((1-(diameter/dn_down)**2)**2)
  bernulli =  (diameter/dn_down)**4 - (diameter/dn_up)**4
  return reducer + diffuser + bernulli

def piping_geometry_factor(f_coefficent, diameter, dn_up, dn_down) -> float:
  '''Return the piping geometry factor fp'''
  diameter *= 1000; dn_up *= 1000; dn_down *= 1000
  rc = resistance_coefficient(diameter/1000, dn_up/1000, dn_down/1000)
  return (1 / np.sqrt(1+(rc*(f_coefficent/diameter**2)**2)/0.0016))
v_piping_geometry_factor = np.vectorize(piping_geometry_factor)

def combined_geometry_factor(f_coefficent, fl, diameter, dn_up, dn_down) -> float:
  ''' Return the Combined liquid pressure recovery factor flp'''
  rc = resistance_coefficient(diameter/1000, dn_up/1000, dn_down/1000)
  return (fl / np.sqrt(1+(rc*(f_coefficent/diameter**2)**2)*(fl**2)/0.0016))
v_combined_geometry_factor = np.vectorize(combined_geometry_factor)

def critical_pressure_factor(temp_c) -> float:
  ''' ff is the Liquid critical pressure ratio factor'''
  pv = v_vapor_pressure(temp_c)
  #  the critical thermodynamic pressure for water is 221.2 bar
  pc = 221.2
  return 0.96-0.28*np.sqrt(pv/pc)
v_critical_pressure_factor = np.vectorize(critical_pressure_factor)

def max_differential_pressure(flp, fp, p1, temp_c) -> float:
  '''The maximum permissible differential pressure'''
  pv = v_vapor_pressure(temp_c)
  ff = v_critical_pressure_factor(temp_c)
  return ((flp/fp)**2)*(p1-ff*pv) 
v_max_differential_pressure = np.vectorize(max_differential_pressure)

#-------------------------------------------------------------------------------
# Plot functions for the Kv/Kvs
def drm_ll3(openinig,b,d,e) -> float:
  return d/(1+np.exp(b*(np.log(openinig)-np.log(e))))

# Solve kv_kvs function
def root_drm_ll3(kv_kvs,b,d,e) -> float:
  def fun(x,kv_kvs,b,d,e):
    return d/(1+np.exp(b*(np.log(x)-np.log(e))))-kv_kvs
  root = fsolve(fun, 50, args=(kv_kvs,b,d,e))
  return root
v_root_drm_ll3 = np.vectorize(root_drm_ll3)

# Plot functions for the Liquid pressure recovery factor Fl
def pressure_recovery_factor(openinig, fls, b, d, e) -> float:
  '''fl The liquid pressure recovery factor'''
  sigma_value = 1/(fls**2) - 1
  kv_kvs = drm_ll3(openinig, b, d, e)
  return np.sqrt(1/(sigma_value * kv_kvs + 1))
v_pressure_recovery_factor = np.vectorize(pressure_recovery_factor)

# plot Cavitation Curves
def cavitation(cav_type, openinig, flps, b, d, e) -> float:
  factor_cav = {'incipient':0.71, 'constant':0.81, 'maximum':1}
  k = factor_cav[cav_type]
  kv_kvs = drm_ll3(openinig, b, d, e)
  return (1/(k * flps**2) - 1) * kv_kvs
#-------------------------------------------------------------------------------

def cavitation_regim(s , sigma_i, sigma_c, sigma_m)-> int:
  if s <= sigma_m:
    return 3
  elif (s > sigma_m) and  (s <= sigma_c):
    return 2
  elif (s > sigma_c) and  (s <= sigma_i):
    return 1
  else:
    return 0
v_cavitation_regim = np.vectorize(cavitation_regim)


# 1. The calculation for a group of operation data

In [ ]:
#@title 1.1 Project Operation data Input
  
# Project Operation data
diameter = 0.200    # meter
dn_up    = 0.200    # meter
dn_down  = 0.200    # meter
o_plate  = 0        # Zeta value of the orifice plate (not yet implemented)
masl     = 1780     # meter
temp_c   = 15       # Celsius
safety_factor = 1.3 # The factor of safety for the Kv, this must be >= 1 

# Operation Data:
#   {'p_up': 'bar', 'p_down': 'bar', 'flow': 'm3/h'}
df2 = np.array(
    [["VRF-15_max",  4.439,	2.615,	396.0],
     ["VRF-16_max",	18.263,	5.241,	396.0],
     ["VRF-18_max",	 9.455,	2.615,	396.0],
     ["VRF-23_max",	20.215,	2.190,	396.0],
     ["VRF-24_max",	19.903,	2.190,	396.0],
     ["VRF-25_max",	19.132,	2.190,	396.0],
     ["VRF-26_max",	18.534,	2.347,	396.0],
     ["VRF-27_max",	17.439,	2.347,	396.0],
     ["VRF-28_max",	17.148,	2.347,	396.0],
     ["VRF-34_max",	18.314,	2.718,	396.0],
     ["VRF-15_min",	 4.368,	2.249,	252.0],
     ["VRF-16_min",	18.193,	5.077,	252.0],
     ["VRF-18_min",	 9.337,	2.249,	252.0],
     ["VRF-23_min",	20.185,	2.081,	252.0],
     ["VRF-24_min",	19.873,	2.081,	252.0],
     ["VRF-25_min",	19.100,	2.081,	252.0],
     ["VRF-26_min",	18.470,	2.148,	252.0],
     ["VRF-27_min",	17.372,	2.148,	252.0],
     ["VRF-28_min",	17.081,	2.148,	252.0],
     ["VRF-34_min",	18.299,	2.291,	252.0]]
  )


In [ ]:
#@title 1.2 Converting the operation data in a Dataframe and print

df2 = (
  pd.DataFrame(df2, columns = ['condition', 'p_up', 'p_down', 'flow'])
  .astype({'condition':'U16', 'p_up':'float64',
           'p_down':'float64', 'flow':'float64'})
)

# Data Table Display
display(df2.round(3))

,condition,p_up,p_down,flow
0,VRF-15_max,4.439,2.615,396.0
1,VRF-16_max,18.263,5.241,396.0
2,VRF-18_max,9.455,2.615,396.0
3,VRF-23_max,20.215,2.190,396.0
4,VRF-24_max,19.903,2.190,396.0
5,VRF-25_max,19.132,2.190,396.0
6,VRF-26_max,18.534,2.347,396.0
7,VRF-27_max,17.439,2.347,396.0
8,VRF-28_max,17.148,2.347,396.0
9,VRF-34_max,18.314,2.718,396.0


In [ ]:
#@title 1.3 Operation Data Processingt

df2 = (
    df2
    .assign(
        p1       = v_absolute_pressure(df2.p_up, masl),
        p2       = v_absolute_pressure(df2.p_down, masl),
        dp       = df2.p_up-df2.p_down,
        velocity = v_velocity(df2.flow, diameter),
        v_factor = velocity_factor(df2.flow, diameter),
        sigma_0  = v_sigma_0(df2.p_up, df2.p_down, masl, temp_c),
        sigma_1  = v_sigma_1(df2.p_up, df2.p_down, masl, temp_c),
        sigma_2  = sigma_2(df2.p_up, df2.p_down, df2.flow, diameter, masl, temp_c),
        kv       = v_flow_coefficent(df2.p_up, df2.p_down, df2.flow, temp_c),
        zeta     = v_drop_coefficient(df2.p_up, df2.p_down, df2.flow, diameter, temp_c)
    )
)

# Data Table Display 
display(df2.round(3))


,condition,p_up,p_down,flow,p1,p2,dp,velocity,v_factor,sigma_0,sigma_1,sigma_2,kv,zeta
0,VRF-15_max,4.439,2.615,396.0,5.256,3.432,1.824,3.501,0.063,2.872,1.872,1.810,293.974,29.561
1,VRF-16_max,18.263,5.241,396.0,19.080,6.058,13.022,3.501,0.063,1.464,0.464,0.462,110.023,211.044
2,VRF-18_max,9.455,2.615,396.0,10.272,3.432,6.840,3.501,0.063,1.499,0.499,0.495,151.807,110.854
3,VRF-23_max,20.215,2.190,396.0,21.032,3.007,18.025,3.501,0.063,1.166,0.166,0.165,93.515,292.126
4,VRF-24_max,19.903,2.190,396.0,20.720,3.007,17.713,3.501,0.063,1.169,0.169,0.168,94.335,287.070
5,VRF-25_max,19.132,2.190,396.0,19.949,3.007,16.942,3.501,0.063,1.176,0.176,0.176,96.458,274.574
6,VRF-26_max,18.534,2.347,396.0,19.351,3.164,16.187,3.501,0.063,1.194,0.194,0.194,98.682,262.338
7,VRF-27_max,17.439,2.347,396.0,18.256,3.164,15.092,3.501,0.063,1.209,0.209,0.208,102.199,244.592
8,VRF-28_max,17.148,2.347,396.0,17.965,3.164,14.801,3.501,0.063,1.213,0.213,0.212,103.199,239.876
9,VRF-34_max,18.314,2.718,396.0,19.131,3.535,15.596,3.501,0.063,1.226,0.226,0.225,100.534,252.760


In [ ]:
#@title 1.4 Reading and preselection of the valves (Brand = "VAG")

# Read the parameter of the VAG Valves.
valves = pd.read_csv(path_valves_data) 

# filter VAG brand valves 
valves = valves[valves.brand == 'VAG']

# select specific columns
columns = ['cyl_name','kv_b','kv_d','kv_e','zvs','fls']
valves = valves[columns]


In [ ]:
#@title 1.5 calculation of Zvs, Kvs, Flps for all possible valves 

#-------------------------------------------------------------------------------
# 1. Calculation of the resistance coefficient of the piping
# 2. resistance coefficient of the valve (Zeta Value) PLUS the piping coefficient
# 3. Calculation of the maximal flow coefficent kvs for all valves

# The new Zeta value is the algebraic sum of all effective resistance 
# coefficients of all fittings attached to the control valve plus the
# resistance coefficient of the control valve.!!!!

valves = (
    valves
    .assign(
        r_coeff = resistance_coefficient(diameter, dn_up, dn_down),
        zvs = valves.zvs + resistance_coefficient(diameter, dn_up, dn_down)
    )
)

# Calculation of the Kvs with the Zvs (Zvs + piping coefficient)
valves = (
    valves
    .assign(
        kvs = v_kv_fun_zeta(diameter, valves.zvs)
    )
)

# Combined liquid pressure recovery factor for full open (Flps)
valves = (
    valves
    .assign(
        flps = v_combined_geometry_factor(
            valves.kvs, valves.fls, 
            diameter, dn_up, dn_down
        )
    )
)

#-------------------------------------------------------------------------------

# 1. Calculating the maximum operating flow coefficient (kv) plus the safety factor. 
# 2. The maximum operating flow coefficient will be the minimum the valve must have.
# 3. Selection of valves that meet the minimum flow coefficient required.
valves = valves.loc[valves.kvs >= (df2.kv.max() * safety_factor)]

# Display
display(valves.round(3))


,cyl_name,kv_b,kv_d,kv_e,zvs,fls,r_coeff,kvs,flps
0,E,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
7,SZ-45,-1.886,5.481,221.491,17.009,0.763,0.0,387.547,0.763
26,L-45 + E,-3.189,1.203,60.647,17.009,0.769,0.0,387.547,0.769
33,S-45 + E,-1.965,16.216,399.724,17.009,0.769,0.0,387.547,0.769
48,SZ-10-30%,-4.271,1.580,88.029,5.560,0.727,0.0,677.847,0.727
49,SZ-10-50%,-5.433,1.362,82.959,8.966,0.837,0.0,533.789,0.837
50,SZ-30-20%,-3.907,1.140,60.456,3.144,0.711,0.0,901.421,0.711
51,SZ-30-50%,-4.670,1.235,73.353,8.104,0.801,0.0,561.460,0.801
53,LH-15-50%,-3.102,79.694,408.503,9.700,0.723,0.0,513.196,0.723
54,LH-20-50%,-3.102,79.694,408.503,5.400,0.723,0.0,687.815,0.723


In [ ]:
#@title 1.5 Nest Operation Data with selected valves 
valves['key'] = 1; df2['key'] = 1
operation_data = pd.merge(valves, df2, on ='key').drop(columns=['key'])

display(operation_data.round(3))


,cyl_name,kv_b,kv_d,kv_e,zvs,fls,r_coeff,kvs,flps,condition,...,p1,p2,dp,velocity,v_factor,sigma_0,sigma_1,sigma_2,kv,zeta
0,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-15_max,...,5.256,3.432,1.824,3.501,0.063,2.872,1.872,1.810,293.974,29.561
1,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-16_max,...,19.080,6.058,13.022,3.501,0.063,1.464,0.464,0.462,110.023,211.044
2,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-18_max,...,10.272,3.432,6.840,3.501,0.063,1.499,0.499,0.495,151.807,110.854
3,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-23_max,...,21.032,3.007,18.025,3.501,0.063,1.166,0.166,0.165,93.515,292.126
4,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-24_max,...,20.720,3.007,17.713,3.501,0.063,1.169,0.169,0.168,94.335,287.070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-25_min,...,19.917,2.898,17.019,2.228,0.025,1.169,0.169,0.169,61.243,681.112
236,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-26_min,...,19.287,2.965,16.322,2.228,0.025,1.181,0.181,0.180,62.537,653.218
237,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-27_min,...,18.189,2.965,15.224,2.228,0.025,1.194,0.194,0.193,64.753,609.275
238,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-28_min,...,17.898,2.965,14.933,2.228,0.025,1.197,0.197,0.197,65.381,597.629


In [ ]:
#@title 1.6 Columns calculation

operation_data = (
    operation_data
    .assign(kv_kvs = operation_data.kv/operation_data.kvs)
)

operation_data = (
    operation_data
    .assign(
        position = v_root_drm_ll3(
          operation_data.kv_kvs, operation_data.kv_b,
          operation_data.kv_d, operation_data.kv_e
        )
    )
)


operation_data = (
    operation_data
    .assign(
        sigma_i = cavitation(
          'incipient', operation_data.position, 
          operation_data.flps, operation_data.kv_b,
          operation_data.kv_d, operation_data.kv_e
        ),
        sigma_c = cavitation(
          'constant', operation_data.position, 
          operation_data.flps, operation_data.kv_b,
          operation_data.kv_d, operation_data.kv_e
        ),
        sigma_m = cavitation(
          'maximum', operation_data.position, 
          operation_data.flps, operation_data.kv_b,
          operation_data.kv_d, operation_data.kv_e
        )
      
    )
)

operation_data = (
    operation_data
    .assign(
        regime = v_cavitation_regim(operation_data.sigma_2,
          operation_data.sigma_i,
          operation_data.sigma_c, 
          operation_data.sigma_m)
    )
)

display(operation_data.round(3))

,cyl_name,kv_b,kv_d,kv_e,zvs,fls,r_coeff,kvs,flps,condition,...,sigma_1,sigma_2,kv,zeta,kv_kvs,position,sigma_i,sigma_c,sigma_m,regime
0,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-15_max,...,1.872,1.810,293.974,29.561,0.254,46.283,0.683,0.568,0.412,0
1,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-16_max,...,0.464,0.462,110.023,211.044,0.095,31.779,0.256,0.212,0.154,0
2,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-18_max,...,0.499,0.495,151.807,110.854,0.131,35.785,0.353,0.293,0.213,0
3,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-23_max,...,0.166,0.165,93.515,292.126,0.081,29.960,0.217,0.181,0.131,2
4,E,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617,VRF-24_max,...,0.169,0.168,94.335,287.070,0.081,30.055,0.219,0.182,0.132,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-25_min,...,0.169,0.169,61.243,681.112,0.063,40.851,0.107,0.086,0.058,0
236,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-26_min,...,0.181,0.180,62.537,653.218,0.064,41.127,0.109,0.088,0.059,0
237,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-27_min,...,0.194,0.193,64.753,609.275,0.067,41.592,0.113,0.091,0.061,0
238,LH-30-50%,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723,VRF-28_min,...,0.197,0.197,65.381,597.629,0.067,41.722,0.114,0.092,0.061,0


In [ ]:
#@title 1.7 splitng the Data Frame

#operation_data = operation_data[[ 
#    'cyl_name','condition', 'kv_b', 'kv_d', 'kv_e', 
#    'zvs', 'fls', 'r_coeff', 'kvs', 'flps', 
#    'p1', 'p2', 'dp', 'flow',  'velocity', 'v_factor', 
#    'sigma_0', 'sigma_1', 'sigma_2', 'kv', 'zeta', 'kv_kvs', 
#    'position', 'sigma_i', 'sigma_c', 'sigma_m', 'regime']]

df2 = operation_data[[ 
    'cyl_name','condition', 'p1', 'p2', 'dp', 'flow', 'velocity',
    'kv', 'kv_kvs', 'zeta', 'sigma_2', 'position', 
    'sigma_i', 'sigma_c', 'sigma_m', 'regime']]

valves =  operation_data[[ 
    'cyl_name','condition', 'kv_b', 'kv_d', 'kv_e', 
    'zvs', 'fls', 'r_coeff', 'kvs', 'flps']]



display(df2.round(3))

print(f'\n')

display(valves.round(3))


,cyl_name,condition,p1,p2,dp,flow,velocity,kv,kv_kvs,zeta,sigma_2,position,sigma_i,sigma_c,sigma_m,regime
0,E,VRF-15_max,5.256,3.432,1.824,396.0,3.501,293.974,0.254,29.561,1.810,46.283,0.683,0.568,0.412,0
1,E,VRF-16_max,19.080,6.058,13.022,396.0,3.501,110.023,0.095,211.044,0.462,31.779,0.256,0.212,0.154,0
2,E,VRF-18_max,10.272,3.432,6.840,396.0,3.501,151.807,0.131,110.854,0.495,35.785,0.353,0.293,0.213,0
3,E,VRF-23_max,21.032,3.007,18.025,396.0,3.501,93.515,0.081,292.126,0.165,29.960,0.217,0.181,0.131,2
4,E,VRF-24_max,20.720,3.007,17.713,396.0,3.501,94.335,0.081,287.070,0.168,30.055,0.219,0.182,0.132,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,LH-30-50%,VRF-25_min,19.917,2.898,17.019,252.0,2.228,61.243,0.063,681.112,0.169,40.851,0.107,0.086,0.058,0
236,LH-30-50%,VRF-26_min,19.287,2.965,16.322,252.0,2.228,62.537,0.064,653.218,0.180,41.127,0.109,0.088,0.059,0
237,LH-30-50%,VRF-27_min,18.189,2.965,15.224,252.0,2.228,64.753,0.067,609.275,0.193,41.592,0.113,0.091,0.061,0
238,LH-30-50%,VRF-28_min,17.898,2.965,14.933,252.0,2.228,65.381,0.067,597.629,0.197,41.722,0.114,0.092,0.061,0


,cyl_name,condition,kv_b,kv_d,kv_e,zvs,fls,r_coeff,kvs,flps
0,E,VRF-15_max,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617
1,E,VRF-16_max,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617
2,E,VRF-18_max,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617
3,E,VRF-23_max,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617
4,E,VRF-24_max,-2.926,1.527,80.354,1.9,0.617,0.0,1159.557,0.617
...,...,...,...,...,...,...,...,...,...,...
235,LH-30-50%,VRF-25_min,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723
236,LH-30-50%,VRF-26_min,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723
237,LH-30-50%,VRF-27_min,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723
238,LH-30-50%,VRF-28_min,-3.102,79.694,408.503,2.7,0.723,0.0,972.718,0.723


In [ ]:
#@title 1.8 Indexing and summarizing the results
ser = df2.set_index(["cyl_name"]). \
      sort_index(level=["cyl_name"])

ser = (
  ser
  .groupby("cyl_name")[["regime","position", "kv_kvs"]]
  .agg([min,max, mean])
)

# Flatten MultiIndex Columns into a Single Index
# https://www.pauldesalvo.com/how-to-flatten-multiindex-columns-into-a-single-index-dataframe-in-pandas/
ser.columns = ['_'.join(col) for col in ser.columns.values]

ser = (
  ser
  .assign(
      kv_kvs_min = ser.kv_kvs_min*100,
      kv_kvs_max = ser.kv_kvs_max*100,
      position_range = ser.position_max - ser.position_min,
      kv_kvs_range = (ser.kv_kvs_max - ser.kv_kvs_min)*100
  )
)


ser = (
  ser
  .assign(
      valve_range =  np.absolute(ser.kv_kvs_range / ser.position_range-1)
  )
)

ser = ser[["regime_min", "regime_max", "regime_mean",
           "position_min", "position_max", "kv_kvs_min", "kv_kvs_max", 
           "position_range", "kv_kvs_range", "valve_range"]]

ser = ser.sort_values(by=["regime_mean", "valve_range"],
                      ascending = [True, True])

display(ser)

,regime_min,regime_max,regime_mean,position_min,position_max,kv_kvs_min,kv_kvs_max,position_range,kv_kvs_range,valve_range
cyl_name,,,,,,,,,,
LH-30-50%,0,0,0.00,40.445473,67.800878,6.104532,30.221907,27.355405,24.117375,0.118369
SZ-10-50%,0,1,0.15,53.141505,77.250239,11.124227,55.073078,24.108734,43.948851,0.822943
SZ-30-20%,0,1,0.20,29.589595,47.838931,6.587365,32.612287,18.249336,26.024921,0.426075
LH-25-50%,0,1,0.25,42.554400,71.348118,7.146137,35.378616,28.793718,28.232479,0.019492
SZ-30-50%,0,1,0.30,44.176324,68.692618,10.575970,52.358802,24.516294,41.782832,0.704288
E,0,2,0.50,25.477249,46.283453,5.120912,25.352266,20.806204,20.231354,0.027629
LH-20-50%,0,2,0.55,45.230973,75.853879,8.633112,42.740231,30.622906,34.107119,0.113778
SZ-10-30%,0,2,0.55,45.322574,70.111699,8.760076,43.368797,24.789125,34.608721,0.396125
LH-15-50%,0,3,1.15,49.715240,83.413593,11.570613,57.283013,33.698353,45.712400,0.356517
